# Playing with OWLready (`owlready2`)

In [1]:
# Install owlready2
# !pip install owlready2

In [2]:
# Import essential modules
import datetime

from owlready2 import *

# Your First OWL Ontology

In [3]:
# Create a new ontology
onto = get_ontology("http://example.org/music.owl")

In [4]:
# Define classes using Python class syntax
with onto:

    class Song(Thing):
        pass

    class Artist(Thing):
        pass

    # Define properties as Python properties
    class performedBy(ObjectProperty):
        domain = [Song]
        range = [Artist]

    class title(DataProperty):
        domain = [Song]
        range = [str]

    class title(AnnotationProperty):
        pass

    class comment(AnnotationProperty):
        pass

In [5]:
# Create instances
bohemian_rhapsody = Song("BohemianRhapsody")
queen = Artist("Queen")

In [6]:
# Set properties using Python assignment
bohemian_rhapsody.title = ["Bohemian Rhapsody"]
bohemian_rhapsody.performedBy = [queen]

In [7]:
print(f"Song: {bohemian_rhapsody.title[0]}")
print(f"Performed by: {bohemian_rhapsody.performedBy[0]}")

Song: Bohemian Rhapsody
Performed by: music.Queen


In [8]:
# dir(onto)

In [9]:
# dir(bohemian_rhapsody)

# Namespaces

In [10]:
# Set ontology metadata
onto.metadata.title = ["Music Ontology"]
onto.metadata.comment = ["An ontology for music domain modeling"]

# Access the ontology's namespace
print(f"Ontology IRI: {onto.base_iri}")
print(f"All classes: {list(onto.classes())}")
print(f"All properties: {list(onto.properties())}")

Ontology IRI: http://example.org/music.owl#
All classes: [music.Song, music.Artist]
All properties: [music.title, music.performedBy, music.comment]


## Sidebar - Merging 2 ontologies

Skip in the first go...   
...or maybe don't  
_(ask the audience)_

Let's bring [FOAF - Friend Of A Friend - Ontoglogy](https://en.wikipedia.org/wiki/FOAF) into the mix.  

Ref:  
* [FOAF @ Linked Open Vocabularies](https://lov.linkeddata.es/dataset/lov/vocabs/foaf)
* Interesting - [foafPub Dataset](https://ebiquity.umbc.edu/resource/html/id/82/)

In [11]:
# You can also work with imported ontologies
foaf = get_ontology("http://xmlns.com/foaf/0.1/").load()
# Load FOAF ontology from web
try:
    foaf = get_ontology("http://xmlns.com/foaf/0.1/").load()
    print("FOAF ontology loaded successfully")

    # Explore FOAF classes
    print("FOAF classes:")
    for cls in foaf.classes():
        print(f"\t{cls.name}")

    # Explore FOAF properties
    print("FOAF properties:")
    for prop in foaf.properties():
        print(f"\t{prop.name}")

except Exception as e:
    print(f"Failed to load FOAF: {e}")
    # Define minimal FOAF classes locally
    with onto:

        class Person(Thing):
            pass

        class name(DataProperty):
            domain = [Person]
            range = [str]

FOAF ontology loaded successfully
FOAF classes:
	Class
	Person
	Person
	Agent
	Agent
	Person
	SpatialThing
	Document
	Organization
	Project
	Document
	Organization
	Group
	Agent-3
	Project
	Image
	PersonalProfileDocument
	OnlineAccount
	OnlineGamingAccount
	OnlineEcommerceAccount
	OnlineChatAccount
FOAF properties:
	mbox_sha1sum
	gender
	geekcode
	dnaChecksum
	sha1
	title
	nick
	jabberID
	aimChatID
	icqChatID
	yahooChatID
	msnChatID
	name
	firstName
	givenname
	surname
	family_name
	plan
	accountName
	birthday
	mbox
	based_near
	phone
	homepage
	page
	weblog
	tipjar
	made
	maker
	img
	depiction
	depicts
	thumbnail
	myersBriggs
	workplaceHomepage
	workInfoHomepage
	schoolHomepage
	knows
	interest
	topic_interest
	publications
	currentProject
	pastProject
	fundedBy
	logo
	topic
	primaryTopic
	theme
	holdsAccount
	accountServiceHomepage
	member
	assurance
	src_assurance
	term_status
	description
	title
	date
	membershipClass


* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/name belongs to more than one entity types: [owl.DatatypeProperty, rdf-schema.label]; I'm trying to fix it...


### heed that warning 
  
So classes have a 'name', properties have a 'name' and the property named 'name' has a name that is 'name'.  
  
One reason we see just a list of properties and not class-wise properties (for those coming from an Object Oriented Programming experience) is that we are trying to build a *vocabulary* for the *world*. That means every word is unique and uniquely suited to one thing only. 
  

 
IMHO - this is also slightly misaligned with how humans "think" in language, where we are able to resolve 'context' from a communication (text, audio, video etc.), thus a bit of ambiguity in words actually results in a more expressive, richer language (for e.g. Poetry). This one-thing-for-one-thing thing is a very knowledge 'engineering' thing. I see a lot of effort now in resolving this, but for our 'context' here, let's just mod things to keep 'em unique. 

In [12]:
# Use FOAF in music ontology
onto.imported_ontologies.append(foaf)

In [13]:
# Now FOAF classes are available in your ontology context
with onto:
    # Can directly use FOAF classes
    class Musician(foaf.Person):
        pass

In [14]:
# Create instances using FOAF classes
freddie = onto.Musician("FreddieMercury")
freddie.foaf_name = ["Freddie Mercury"]  # Use FOAF properties

In [15]:
# check if freddie.name is defined, else print 'No Name'
print(f"Musician created: {freddie.name if freddie.name else 'No name'}")

Musician created: FreddieMercury


In [16]:
onto.get_imported_ontologies()

[get_ontology("http://xmlns.com/foaf/0.1/")]

In [17]:
for p in onto.metadata:
    print(p)

None
music.title
music.comment


## Default World

_(Gruff trailer voice) IN A WORLD..._  
  
Vocabularies belong to a world.  
owlready2 has a 'default' world.  
You can add your own 'world' too.

In [18]:
# default_world is owlready2's global knowledge base container
# It automatically stores ALL loaded ontologies and their instances
# Think of it as a single RDF graph containing everything you've loaded

When you do this:

In [19]:
music_onto = get_ontology("./rdf_output/music_ontology.rdf").load()
foaf = get_ontology("http://xmlns.com/foaf/0.1/").load()

Both ontologies get added to default_world automatically
`default_world` now contains:
  - All classes from `music_onto` (Song, Artist, Album, etc.)
  - All properties from `music_onto` (performedBy, hasGenre, etc.) 
  - All instances you create (queen, bohemian_rhapsody, etc.)
  - All classes/properties from `foaf` (Person, knows, etc.)

In [20]:
# Example: default_world contains everything
print("What's in my default_world?")
print(f"Total ontologies loaded: {len(default_world.ontologies)}")
print(
    f"Total individuals across all ontologies: {len(list(default_world.individuals()))}"
)
print(f"Total classes across all ontologies: {len(list(default_world.classes()))}")

What's in my default_world?
Total ontologies loaded: 5
Total individuals across all ontologies: 3
Total classes across all ontologies: 35


In [21]:
# You can also access specific ontologies within default_world
for o in default_world.ontologies:
    print(f"Ontology: {o}")

Ontology: http://anonymous/
Ontology: http://example.org/music.owl#
Ontology: http://xmlns.com/foaf/0.1/
Ontology: ./rdf_output/music_ontology.rdf#
Ontology: http://example.org/music#


Alternative worlds: You can create separate knowledge bases  

In [22]:
# custom_world = World()  # Creates isolated knowledge base
# with custom_world:
#     specific_onto = get_ontology("./o02-Music.n3").load()
# custom_world.sparql(query)  # Only searches custom_world, not default_world

_Caution: default_world is global and persistent_  
All ontologies you load stay in memory until you explicitly remove them    
Use `onto.destroy()` to remove ontologies from `default_world` - super critical to clean up...   

# Loading External RDF/OWL Data

In [23]:
# Load the existing music ontology
music_onto = get_ontology("./rdf_output/music_ontology.rdf").load()

In [24]:
# Explore loaded classes
print("Classes in the ontology:")
for cls in music_onto.classes():
    print(f"  {cls.name}:\t{cls.comment}")

Classes in the ontology:
  Artist:	[]
  CollaborativeSong:	[]
  Song:	[]
  Award:	[]
  Album:	[]
  Genre:	[]
  Single:	[]
  SuccessfulLabel:	[]
  RecordLabel:	[]
  ExtendedPlay:	[]
  EstablishedArtist:	[]


In [25]:
# Explore properties
print("\nProperties in the ontology:")
for prop in music_onto.properties():
    print(f"\t{prop.name}: domain={prop.domain}, range={prop.range}")


Properties in the ontology:
	popularityScore: domain=[], range=[<class 'int'>]
	nationality: domain=[], range=[<class 'str'>]
	description: domain=[], range=[<class 'str'>]
	location: domain=[], range=[<class 'str'>]
	labelName: domain=[], range=[<class 'str'>]
	title: domain=[], range=[<class 'str'>]
	birthDate: domain=[], range=[<class 'datetime.date'>]
	awardName: domain=[], range=[<class 'str'>]
	collaborationStrength: domain=[], range=[<class 'int'>]
	name: domain=[], range=[<class 'str'>]
	awardingBody: domain=[], range=[<class 'str'>]
	releaseYear: domain=[], range=[<class 'int'>]
	labelSuccessRating: domain=[], range=[<class 'int'>]
	albumTitle: domain=[], range=[<class 'str'>]
	year: domain=[], range=[<class 'int'>]
	duration: domain=[], range=[<class 'int'>]
	genreName: domain=[], range=[<class 'str'>]
	releaseDate: domain=[], range=[<class 'datetime.date'>]
	collaboratesWith: domain=[], range=[]
	influencedBy: domain=[], range=[]
	performedBy: domain=[], range=[]
	hasGenre:

In [26]:
# Check reasoner-inferred classes
print(f"\nTotal classes: {len(list(music_onto.classes()))}")
print(f"Total properties: {len(list(music_onto.properties()))}")


Total classes: 11
Total properties: 29


# Basic Queries

owlready2 offers two query approaches.   
* Python iteration and comprehensions work well for simple queries.
* SPARQL handles complex aggregations that would be harder to maintain in pure Python.

In [27]:
# Helper function
def format_sparql_results(results, query_description=""):
    print(f"\n{query_description}")
    print("=" * len(query_description))
    #
    results_list = list(results)
    if not results_list:
        print("No results found.")
        return
    #
    # Get variable names from first result
    if hasattr(results_list[0], "__dict__"):
        # print('__dict__ found in results')
        headers = list(results_list[0].__dict__.keys())
    else:
        headers = [f"col_{i}" for i in range(len(results_list[0]))]
    # print ('headers = ',headers)
    #
    # Format headers
    print(" | ".join(f"{h:20}" for h in headers))
    print("-" * (23 * len(headers) - 3))

    # Format rows
    for row in results_list[:10]:  # Limit to first 10 results
        formatted_row = []
        for item in row:
            if item is None:
                formatted_row.append("None")
            elif hasattr(item, "split") and "#" in str(item):
                # Extract local name from URI
                formatted_row.append(str(item).split("#")[-1])
            elif hasattr(item, "split") and "/" in str(item):
                # Extract last part of path
                formatted_row.append(str(item).split("/")[-1])
            else:
                formatted_row.append(str(item))
        print(" | ".join(f"{item:20}" for item in formatted_row))

    if len(results_list) > 10:
        print(f"... and {len(results_list) - 10} more results")


# Usage:
# format_sparql_results(results, description)

# Query 1: Find all songs and their performers  
...using Python iteration

In [28]:
print("Query 1: Songs and performers")
out = []
for song in music_onto.Song.instances():
    if hasattr(song, "performedBy") and song.performedBy:
        for performer in song.performedBy:
            out.append(f"{song.name} performed by {performer.name}")
out[:10]

Query 1: Songs and performers


['song_song_421 performed by artist_artist_35',
 'song_song_421 performed by artist_artist_20',
 'song_song_386 performed by artist_artist_20',
 'song_song_386 performed by artist_artist_50',
 'song_song_541 performed by artist_artist_19',
 'song_song_541 performed by artist_artist_56',
 'song_song_244 performed by artist_artist_23',
 'song_song_244 performed by artist_artist_25',
 'song_song_11 performed by artist_artist_90',
 'song_song_11 performed by artist_artist_75']

# Query 2: Find artists signed to labels  
using property filtering

In [29]:
print("\nQuery 2: Artists and their labels")
out = []
for artist in music_onto.Artist.instances():
    if hasattr(artist, "signedTo") and artist.signedTo:
        out.append(f"{artist.name} signed to {artist.signedTo[0].name}")
out[:10]


Query 2: Artists and their labels


[]

## Sidebar - looking deeper into Query 2
_skip in the first iteration_  
Q2 results are empty!  
What's going on?  
Let's try to debug this - see what's really going on...

In [30]:
# Reason 1: No Artist instances exist
artists = list(music_onto.Artist.instances())
# quick refresher - what is a label called in our ontology?
# check back - we listed all the classes in our ontology...
labels = list(music_onto.RecordLabel.instances())
print(
    f"1. Artist instances found: {len(artists)} and RecordLabel instances found: {len(labels)}"
)

1. Artist instances found: 100 and RecordLabel instances found: 0


How come we have no RecordLabel associations???  
Goes back to the reasoner... when creating this data, we rely on `signed_artists` inverse property...
...ah! that's still TODO!

# Query 3: Number of Songs by Genre
Use SPARQL for complex aggregation queries

## Quick primer on SPARQL in `owlready2`

SPARQL queries through default_world search across ALL loaded ontologies  
This is why we use: default_world.sparql(query)  
Instead of: music_onto.sparql(query) - which may not exist (since we are mixing stuff up now...)

Why default_world matters for SPARQL:
1. SPARQL needs a complete RDF graph to query against
2. default_world automatically maintains this graph
3. All your loaded data is queryable in one place
4. No need to manually merge ontologies for cross-ontology queries

In [31]:
# SPARQL queries search the entire default_world knowledge base
# This query will find instances from ANY loaded ontology
sparql_query = """
PREFIX music: <http://example.org/music#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?song ?performer WHERE {
    ?song rdf:type music:Song .
    ?song music:performedBy ?performer .
}
"""

In [32]:
# This searches ALL RDF triples in default_world (from all ontologies)
results = default_world.sparql(sparql_query)

In [33]:
format_sparql_results(results, "Working with default_world")


Working with default_world
col_0                | col_1               
-------------------------------------------
music.song_song_391  | music.artist_artist_36
music.song_song_444  | music.artist_artist_43
music.song_song_518  | music.artist_artist_76
music.song_song_285  | music.artist_artist_55
music.song_song_100  | music.artist_artist_76
music.song_song_296  | music.artist_artist_80
music.song_song_155  | music.artist_artist_13
music.song_song_38   | music.artist_artist_29
music.song_song_459  | music.artist_artist_89
music.song_song_481  | music.artist_artist_0
... and 288 more results


## ...let's build the actual query now: count songs by genre

In [34]:
# as a quick refresher, let's just list all the properties in our music ontology
for cls in music_onto.classes():
    print(f"{cls.name}:\t{cls.iri}")

Artist:	http://example.org/music#Artist
CollaborativeSong:	http://example.org/music#CollaborativeSong
Song:	http://example.org/music#Song
Award:	http://example.org/music#Award
Album:	http://example.org/music#Album
Genre:	http://example.org/music#Genre
Single:	http://example.org/music#Single
SuccessfulLabel:	http://example.org/music#SuccessfulLabel
RecordLabel:	http://example.org/music#RecordLabel
ExtendedPlay:	http://example.org/music#ExtendedPlay
EstablishedArtist:	http://example.org/music#EstablishedArtist


In [35]:
# as a quick refresher, let's just list all the properties in our music ontology
for property in music_onto.properties():
    print(f"{property.name}:\t{property.iri}")

popularityScore:	http://example.org/music#popularityScore
nationality:	http://example.org/music#nationality
description:	http://example.org/music#description
location:	http://example.org/music#location
labelName:	http://example.org/music#labelName
title:	http://example.org/music#title
birthDate:	http://example.org/music#birthDate
awardName:	http://example.org/music#awardName
collaborationStrength:	http://example.org/music#collaborationStrength
name:	http://example.org/music#name
awardingBody:	http://example.org/music#awardingBody
releaseYear:	http://example.org/music#releaseYear
labelSuccessRating:	http://example.org/music#labelSuccessRating
albumTitle:	http://example.org/music#albumTitle
year:	http://example.org/music#year
duration:	http://example.org/music#duration
genreName:	http://example.org/music#genreName
releaseDate:	http://example.org/music#releaseDate
collaboratesWith:	http://example.org/music#collaboratesWith
influencedBy:	http://example.org/music#influencedBy
performedBy:	h

So classes of interest: `Song` and `Genre`, property that connects them: `hasGenre`

In [36]:
# we need to define what "music" is for SPARQL, so we'll use PREFIX
sparql_query = """
PREFIX music: <http://example.org/music#>
SELECT ?genre (COUNT(?song) as ?count) WHERE {
    ?song rdf:type music:Song .
    ?song music:hasGenre ?genre .
}
GROUP BY ?genre
ORDER BY DESC(?count)
"""

In [37]:
# now we query SPARQL through the default_world
print("\nQuery 3: Songs by genre (using SPARQL)")

results = default_world.sparql(sparql_query)
for genre, count in results:
    print(f"  Genre: {genre}, Songs: {count}")


Query 3: Songs by genre (using SPARQL)
  Genre: music.genre_genre_4, Songs: 39
  Genre: music.genre_genre_0, Songs: 39
  Genre: music.genre_genre_9, Songs: 37
  Genre: music.genre_genre_13, Songs: 33
  Genre: music.genre_genre_6, Songs: 32
  Genre: music.genre_genre_12, Songs: 28
  Genre: music.genre_genre_10, Songs: 27
  Genre: music.genre_genre_1, Songs: 27
  Genre: music.genre_genre_7, Songs: 23
  Genre: music.genre_genre_14, Songs: 22
  Genre: music.genre_genre_11, Songs: 21
  Genre: music.genre_genre_8, Songs: 19
  Genre: music.genre_genre_3, Songs: 18
  Genre: music.genre_genre_2, Songs: 16
  Genre: music.genre_genre_5, Songs: 11


# Query 4: Find collaborative songs  
...using list comprehension

In [38]:
print("\nQuery 4: Collaborative songs")
collaborative_songs = [
    song
    for song in music_onto.Song.instances()
    if hasattr(song, "performedBy") and len(song.performedBy) > 1
]

out = []
for song in collaborative_songs:
    performers = [p.name for p in song.performedBy]
    out.append(f"  {song.name}: {', '.join(performers)}")
out[:10]


Query 4: Collaborative songs


['  song_song_421: artist_artist_35, artist_artist_20',
 '  song_song_386: artist_artist_20, artist_artist_50',
 '  song_song_541: artist_artist_19, artist_artist_56',
 '  song_song_244: artist_artist_23, artist_artist_25',
 '  song_song_11: artist_artist_90, artist_artist_75',
 '  song_song_419: artist_artist_56, artist_artist_58',
 '  song_song_316: artist_artist_83, artist_artist_22',
 '  song_song_319: artist_artist_85, artist_artist_77',
 '  song_song_404: artist_artist_29, artist_artist_84',
 '  song_song_60: artist_artist_26, artist_artist_57']

# Query 5: Albums and track counts 
...using Python aggregation

In [39]:
print("\nQuery 5: Albums and track counts")
album_tracks = {}
for song in music_onto.Song.instances():
    if hasattr(song, "featuredOn"):
        for album in song.featuredOn:
            album_name = album.name if hasattr(album, "name") else str(album)
            album_tracks[album_name] = album_tracks.get(album_name, 0) + 1

if len(album_tracks.items()) > 0:
    for album, count in album_tracks.items():
        print(f"{album}: {count} tracks")
else:
    print("No results found...")


Query 5: Albums and track counts
No results found...


## Sidebar - looking deeper into Query 5
_skip in the first iteration_  
Q5 results are empty!  
What's going on?  
Let's try to debug this - see what's really going on...

In [40]:
# Possible Reason 1: There's no songs or albums
songs = list(music_onto.Song.instances())
albums = list(music_onto.Album.instances())

print(f"1. Song instances found: {len(songs)} and Album instances found: {len(albums)}")

1. Song instances found: 600 and Album instances found: 100


In [41]:
# Let's list the properties of a song
if songs:
    sample_song = songs[0]
    song_props = [
        attr for attr in dir(sample_song) if not attr.startswith("_")
    ]  # omit the internals
    print(f"Sample song properties: {song_props}")

Sample song properties: ['INDIRECT_get_properties', 'differents', 'duration', 'entity_class', 'featuredOn', 'generate_default_name', 'get_equivalent_to', 'get_inverse_properties', 'get_iri', 'get_name', 'get_properties', 'hasGenre', 'iri', 'is_a', 'is_instance_of', 'name', 'namespace', 'performedBy', 'reload', 'set_equivalent_to', 'set_iri', 'set_name', 'storid', 'title']


In [42]:
# Similarly, let's list the properties of an album
if albums:
    sample_album = albums[0]
    album_props = [attr for attr in dir(sample_album) if not attr.startswith("_")]
    print(f"Sample album properties: {album_props}")

Sample album properties: ['INDIRECT_get_properties', 'albumTitle', 'differents', 'entity_class', 'generate_default_name', 'get_equivalent_to', 'get_inverse_properties', 'get_iri', 'get_name', 'get_properties', 'hasGenre', 'iri', 'is_a', 'is_instance_of', 'name', 'namespace', 'releaseYear', 'reload', 'set_equivalent_to', 'set_iri', 'set_name', 'storid']


Okay, so Albums and Songs connect using the `featuredOn` property in Songs.

In [43]:
if songs:
    # Possible Reason 2: Songs don't have 'featuredOn' property
    songs_with_featured_on = [s for s in songs if hasattr(s, "featuredOn")]
    print(f"2. Songs with 'featuredOn' property: {len(songs_with_featured_on)}")

    # Possible Reason 3: 'featuredOn' property exists but is empty
    songs_with_albums = [s for s in songs_with_featured_on if s.featuredOn]
    print(f"3. Songs with non-empty 'featuredOn': {len(songs_with_albums)}")

2. Songs with 'featuredOn' property: 600
3. Songs with non-empty 'featuredOn': 0


...there we have it, the `featuredOn` property for all songs is empty in our data.

# Creating Music Classes and Properties

In [44]:
# Create extended ontology
extended_onto = get_ontology("http://example.org/music_extended.owl")

In [45]:
with extended_onto:
    # Import base music classes (already a part of music_onto)
    class Song(music_onto.Song):
        pass

    class Album(music_onto.Album):
        pass

    # Define three new classes
    class Playlist(Thing):
        comment = ["A curated collection of songs"]

    class Concert(Thing):
        comment = ["A live music performance event"]

    class MusicVideo(Thing):
        comment = ["A video recording featuring a song"]

## Properties for `Playlist`

In [46]:
with extended_onto:
    # Properties for Playlist
    class containsSong(ObjectProperty):
        domain = [Playlist]
        range = [Song]
        comment = ["Songs included in the playlist"]

    class playlistName(DataProperty):
        domain = [Playlist]
        range = [str]

    class createdBy(ObjectProperty):
        domain = [Playlist]
        # range = [Person]  # Would need FOAF import, maybe later

## Properties for `Concert`

In [47]:
with extended_onto:
    # Properties for Concert
    class performsAt(ObjectProperty):
        domain = [Artist]
        range = [Concert]

    # we'll later see that HermIT reasoner doesn't support datetime.date...
    # so we'll use datetime.datetime everywhere...
    # or better yet - just ISO date strings
    # quirky, I know.
    class concertDate(DataProperty):
        domain = [Concert]
        # range = [datetime.datetime]
        range = [str]

    class venue(DataProperty):
        domain = [Concert]
        range = [str]

## Properties for `MusicVideo`

In [48]:
with extended_onto:
    # Properties for MusicVideo
    class videoFor(ObjectProperty):
        domain = [MusicVideo]
        range = [Song]
        functional = True  # Each video is for exactly one song

    class director(DataProperty):
        domain = [MusicVideo]
        range = [str]

    class viewCount(DataProperty):
        domain = [MusicVideo]
        range = [int]

owlready2's `with onto:` context manager automatically assigns new classes to the ontology.  
Property constraints like `functional = True` are set as Python attributes, making OWL semantics transparent.

# Working with Individuals

In [49]:
# Playlist
with extended_onto:
    # Create specific instances using constructor syntax
    roadtrip_playlist = Playlist("RoadTrip2024")
    roadtrip_playlist.playlistName = ["Road Trip 2024"]

In [50]:
# Concert
with extended_onto:
    wembley_concert = Concert("Wembley1986")
    # wembley_concert.concertDate = [datetime.datetime(1986, 7, 12)]
    wembley_concert.concertDate = ["1986-07-12"]  # ISO date string
    wembley_concert.venue = ["Wembley Stadium"]

In [51]:
# Music Video
with extended_onto:
    bohemian_video = MusicVideo("BohemianRhapsodyVideo")
    bohemian_video.director = ["Bruce Gowers"]
    bohemian_video.viewCount = [1500000000]

In [52]:
# Connecting the 3
with extended_onto:
    # Create relationships
    queen.performsAt = [wembley_concert]
    roadtrip_playlist.containsSong = [bohemian_rhapsody]
    bohemian_video.videoFor = [bohemian_rhapsody]

In [53]:
# Print instance details
print("Created instances:")
for individual in extended_onto.individuals():
    print(f"  {individual.name}: {type(individual).__name__}")

Created instances:
  RoadTrip2024: Playlist
  Wembley1986: Concert
  BohemianRhapsodyVideo: MusicVideo


In [54]:
for p in extended_onto.properties():
    print(p)

music_extended.playlistName
music_extended.concertDate
music_extended.venue
music_extended.director
music_extended.viewCount
music_extended.containsSong
music_extended.createdBy
music_extended.performsAt
music_extended.videoFor


# Possible _Restrictions_ and **Reasoning** for Music Domain

## Gather our bearings

In [55]:
extended_onto.ontology

get_ontology("http://example.org/music_extended.owl#")

In [56]:
extended_onto.imported_ontologies

[]

In [57]:
for o in extended_onto.imported_ontologies:
    print(f"extended_onto.imported_ontologies: {o}")

### Ensure music ontology (the RDF we imported) classes are available to `extended_onto`

In [58]:
# Import music ontology into extended ontology for access to classes
with extended_onto:
    # Make music_onto classes available
    if music_onto not in extended_onto.imported_ontologies:
        extended_onto.imported_ontologies.append(music_onto)

In [59]:
for cls in music_onto.classes():
    print(f"{cls.name}:\t{cls.iri}")

Artist:	http://example.org/music#Artist
CollaborativeSong:	http://example.org/music#CollaborativeSong
Song:	http://example.org/music#Song
Award:	http://example.org/music#Award
Album:	http://example.org/music#Album
Genre:	http://example.org/music#Genre
Single:	http://example.org/music#Single
SuccessfulLabel:	http://example.org/music#SuccessfulLabel
RecordLabel:	http://example.org/music#RecordLabel
ExtendedPlay:	http://example.org/music#ExtendedPlay
EstablishedArtist:	http://example.org/music#EstablishedArtist


In [60]:
for cls in extended_onto.classes():
    print(f"{cls.name}:\t{cls.iri}")

Song:	http://example.org/music_extended.owl#Song
Album:	http://example.org/music_extended.owl#Album
Playlist:	http://example.org/music_extended.owl#Playlist
Concert:	http://example.org/music_extended.owl#Concert
MusicVideo:	http://example.org/music_extended.owl#MusicVideo


## Existential Restrictions (**some**)
_"Has at least one relationship of this type"_

In [61]:
with extended_onto:
    # 1. Award-winning artists: Artists who have won some award
    class AwardWinningArtist(music_onto.Artist):
        comment = ["Artists who have won at least one award"]
        # equivalent_to: Artist AND hasWonAward some Award
        equivalent_to = [
            music_onto.Artist & music_onto.hasWonAward.some(music_onto.Award)
        ]

    # 2. Collaborative artists: Artists who collaborate with some other artist
    class CollaborativeArtist(music_onto.Artist):
        comment = ["Artists who collaborate with at least one other artist"]
        # equivalent_to: Artist AND collaboratesWith some Artist
        equivalent_to = [
            music_onto.Artist & music_onto.collaboratesWith.some(music_onto.Artist)
        ]

    # 3. Label artists: Artists signed to some record label
    class LabelArtist(music_onto.Artist):
        comment = ["Artists signed to at least one record label"]
        # equivalent_to: Artist AND signedTo some RecordLabel
        equivalent_to = [
            music_onto.Artist & music_onto.signedTo.some(music_onto.RecordLabel)
        ]

## Cardinality Restrictions (**exactly, min, max**)
_"Has exactly/at least/at most N relationships"_  
Very much like the One-to-Many, Many-to-One, One-to-One, Many-to-Many cardinality restrictions we see in databases and object-oriented programming


In [62]:
with extended_onto:
    # 4. Solo songs: Songs performed by exactly one artist
    class SoloSong(music_onto.Song):
        comment = ["Songs performed by exactly one artist"]
        # equivalent_to: Song AND performedBy exactly 1 Artist
        equivalent_to = [
            music_onto.Song & music_onto.performedBy.exactly(1, music_onto.Artist)
        ]

    # 5. Multi-artist songs: Songs with more than one performer (using min)
    class MultiArtistSong(music_onto.Song):
        comment = ["Songs performed by multiple artists"]
        # equivalent_to: Song AND performedBy min 2 Artist
        equivalent_to = [
            music_onto.Song & music_onto.performedBy.min(2, music_onto.Artist)
        ]

## Universal Restrictions (**only**)  
_"All relationships of this type must be to this class"_

In [63]:
# not implemented the inverse relations in the reasoner,
# these do not feature in the rdf we imported
# so for now we'll just give examples here.
with extended_onto:
    # 6. Rock-only albums: Albums where all songs are rock genre
    # First, create a RockSong class based on genre
    class RockSong(music_onto.Song):
        comment = ["Songs with rock genre"]
        # We've not defined "RockGenre" etc. either
        # RockSong restriction would be defined by "hasGenre some RockGenre", but we'll keep it simple
        pass

    class RockOnlyAlbum(music_onto.Album):
        comment = ["Albums containing only rock songs"]
        # equivalent_to: Album AND featuredOn only RockSong
        # Note: We use inverse - albums that only feature rock songs
        pass  # Skip this one as featuredOn direction might be complex

## Value Restrictions (*hasValue*)
_"Has a specific individual as the relationship target"_

In [64]:
# skip for now
# with extended_onto:
# 7. Genre-specific restrictions using existing Genre instances
# We'll need specific genre instances defined, fairly straight forward, let's skip for now

## Complex Restrictions (combining multiple constraints)

In [65]:
with extended_onto:
    # 8. Established solo artists: Artists with awards AND no collaborations
    class EstablishedSoloArtist(music_onto.Artist):
        comment = ["Successful artists who work alone"]
        # equivalent_to: Artist AND hasWonAward some Award AND collaboratesWith exactly 0 Artist
        equivalent_to = [
            music_onto.Artist
            & music_onto.hasWonAward.some(music_onto.Award)
            & music_onto.collaboratesWith.exactly(0, music_onto.Artist)
        ]

    # 9. Popular albums: Albums by award-winning artists with multiple songs
    class PopularAlbum(music_onto.Album):
        comment = ["Albums by successful artists with multiple tracks"]
        # This is complex - album released by artist who has awards
        # We'll use a simpler version focusing on the album itself
        pass  # Skip complex cross-property restrictions for now

## Let's test the restrictions now... let the _reasoning_ begin

### create some more sample data

In [66]:
with extended_onto:
    # Create test artists
    artist1 = music_onto.Artist("TestArtist1")
    artist1.name = ["Solo Artist"]

    artist2 = music_onto.Artist("TestArtist2")
    artist2.name = ["Collaborative Artist"]

In [67]:
with extended_onto:
    # Create test award
    award1 = music_onto.Award("TestAward1")
    award1.awardName = ["Best Artist Award"]

In [68]:
with extended_onto:
    # Create test record label
    label1 = music_onto.RecordLabel("TestLabel1")
    label1.labelName = ["Test Records"]

In [69]:
with extended_onto:
    # Create test songs
    solo_song = music_onto.Song("SoloSong1")
    solo_song.title = ["Solo Track"]
    solo_song.performedBy = [artist1]  # Exactly 1 performer -> SoloSong

    collab_song = music_onto.Song("CollabSong1")
    collab_song.title = ["Collaboration Track"]
    collab_song.performedBy = [artist1, artist2]  # 2 performers -> MultiArtistSong

### Set up relationships to trigger restrictions

In [70]:
# Make artist1 an award winner (should become AwardWinningArtist)
artist1.hasWonAward = [award1]

In [71]:
# Make artist2 collaborative (should become CollaborativeArtist)
artist2.collaboratesWith = [artist1]
artist1.collaboratesWith = [artist2]  # Make it bidirectional

In [72]:
# Make artist1 signed to label (should become LabelArtist)
artist1.signedTo = [label1]

In [73]:
for individual in extended_onto.individuals():
    if hasattr(individual, "name") and individual.name:
        # individuals may be of more than one type
        # or type (the 'is_a' property) may not be defined
        current_types = [t.name for t in individual.is_a if hasattr(t, "name")]
        print(f"{individual.name} is a {current_types}")

RoadTrip2024 is a ['Playlist']
Wembley1986 is a ['Concert']
BohemianRhapsodyVideo is a ['MusicVideo']
['Solo Artist'] is a ['Artist']
['Collaborative Artist'] is a ['Artist']
TestAward1 is a ['Award']
TestLabel1 is a ['RecordLabel']
SoloSong1 is a ['Song']
CollabSong1 is a ['Song']


### Use one of the bundeled reasoners: HermiT

_TODO: Build a full separate workshop on reasoners like HermiT.  
This may be confusing (as we are muddling WHAT we want to do - reasoning with ontology, with HOW HermiT works, just more cognitive load and less learning), skip in the first iteration, better to implement a DIY reasoning approach_

In [ ]:
with extended_onto:
    sync_reasoner_hermit(infer_property_values=True)

In [75]:
restriction_classes = [
    "AwardWinningArtist",
    "CollaborativeArtist",
    "LabelArtist",
    "SoloSong",
    "MultiArtistSong",
    "EstablishedSoloArtist",
]

In [76]:
for individual in extended_onto.individuals():
    current_types = [t.name for t in individual.is_a if hasattr(t, "name")]

    # Check if any restriction classes were inferred
    restriction_types = [t for t in current_types if t in restriction_classes]

    if restriction_types:
        print(f"{individual.name}: {current_types}")
        print(f"Triggered restrictions: {restriction_types}")
    else:
        print(f"{individual.name} is a {current_types} (no restrictions triggered)")

RoadTrip2024 is a ['Playlist'] (no restrictions triggered)
Wembley1986 is a ['Concert'] (no restrictions triggered)
BohemianRhapsodyVideo is a ['MusicVideo'] (no restrictions triggered)
['Solo Artist'] is a ['Artist'] (no restrictions triggered)
['Collaborative Artist'] is a ['Artist'] (no restrictions triggered)
TestAward1 is a ['Award'] (no restrictions triggered)
TestLabel1 is a ['RecordLabel'] (no restrictions triggered)
SoloSong1 is a ['Song'] (no restrictions triggered)
CollabSong1 is a ['Song'] (no restrictions triggered)


In [77]:
# Check for inconsistencies
inconsistent = list(default_world.inconsistent_classes())
if inconsistent:
    print(f"\nInconsistent classes: {[cls.name for cls in inconsistent]}")
else:
    print("\nOntology remains consistent")


Ontology remains consistent


### or DIY - no additional reasoner needed  
  
Here's also why this is important: We get to see how the sausage is made.

1. Compare with reasoner results to ensure correctness. 
2. This is also a fallback option: here we show expected results when reasoner fails/unavailable. In some of the cases the reasoner may not be configured correctly or may not be working, we can do the same thing here instead.  
3. DIY helps us easily identify if restrictions are defined correctly
4. SHOW and TELL: Demonstrate the logic behind OWL restrictions in familiar Python

In [78]:
for individual in extended_onto.individuals():
    if not hasattr(individual, "name") or not individual.name:
        continue

    individual_name = individual.name
    # manual_types replicates the LOGIC of OWL restrictions in Python code
    # Each check below mirrors an OWL restriction definition we created earlier
    # This serves as a "ground truth" to verify what the reasoner SHOULD infer
    manual_types = (
        []
    )  # List to collect restriction classes this individual should belong to

    # Check AwardWinningArtist restriction
    # OWL: Artist & hasWonAward.some(Award)
    # Python equivalent: "Is this an Artist AND does it have at least one award?"
    if (
        isinstance(individual, music_onto.Artist)
        and hasattr(individual, "hasWonAward")
        and individual.hasWonAward
    ):
        manual_types.append("AwardWinningArtist")
        # What's going on: We manually check the same condition the OWL reasoner would check
        # If True, this individual should be classified as AwardWinningArtist

    # Check CollaborativeArtist restriction
    # OWL: Artist & collaboratesWith.some(Artist)
    # Python equivalent: "Is this an Artist AND does it collaborate with someone?"
    if (
        isinstance(individual, music_onto.Artist)
        and hasattr(individual, "collaboratesWith")
        and individual.collaboratesWith
    ):
        manual_types.append("CollaborativeArtist")
        # This checks: "Does this artist have at least one collaboration relationship?"

    # Check LabelArtist restriction
    # OWL: Artist & signedTo.some(RecordLabel)
    # Python equivalent: "Is this an Artist AND is it signed to a label?"
    if (
        isinstance(individual, music_onto.Artist)
        and hasattr(individual, "signedTo")
        and individual.signedTo
    ):
        manual_types.append("LabelArtist")
        # We're checking: "Does this artist have a record label contract?"

    # Check SoloSong restriction
    # OWL: Song & performedBy.exactly(1, Artist)
    # Python equivalent: "Is this a Song AND does it have exactly 1 performer?"
    if (
        isinstance(individual, music_onto.Song)
        and hasattr(individual, "performedBy")
        and len(individual.performedBy) == 1
    ):
        manual_types.append("SoloSong")
        # Checks cardinality: exactly one performer, no more, no less

    # Check MultiArtistSong restriction
    # OWL: Song & performedBy.min(2, Artist)
    # Python equivalent: "Is this a Song AND does it have 2+ performers?"
    if (
        isinstance(individual, music_onto.Song)
        and hasattr(individual, "performedBy")
        and len(individual.performedBy) >= 2
    ):
        manual_types.append("MultiArtistSong")
        # This checks minimum cardinality: at least 2 performers

    if manual_types:
        print(f"{individual_name} should be: {manual_types}")
        # This shows what classes the individual SHOULD belong to based on our manual logic
        # If reasoner works correctly, it should infer the same classifications

['Solo Artist'] should be: ['AwardWinningArtist', 'CollaborativeArtist', 'LabelArtist']
['Collaborative Artist'] should be: ['CollaborativeArtist']
SoloSong1 should be: ['SoloSong']
CollabSong1 should be: ['MultiArtistSong']


## Ontology driven vs traditional

### Traditional Database Approach 
In traditional databases, you must EXPLICITLY state every fact:
- INSERT INTO artist_types (artist_id, type) VALUES (1, 'AwardWinning')
- You have to manually maintain all derived data
- If artist wins new award, you must remember to update artist_types table
- No automatic inference - everything is manual labor

### Ontology driven approach (OWL Semantics here) 
In OWL, you define LOGICAL RULES (restrictions) and the reasoner applies them automatically:

1. DEFINE RULES (OWL Restrictions):

   ```python
   class AwardWinningArtist(Artist):
       equivalent_to = [Artist & hasWonAward.some(Award)]
    ```
   This means: "Any individual that is an Artist AND has won some Award 
               is AUTOMATICALLY an AwardWinningArtist"

2. ASSERT FACTS (Instance Data):
   ```python
   freddie = Artist("Freddie") 
   grammy = Award("Grammy")
   freddie.hasWonAward = [grammy]
   ```
   We only state the basic facts, not the derived conclusions

3. REASONER APPLIES LOGIC:
   The reasoner reads the restriction rule and checks all individuals:
   - Is freddie an Artist? YES
   - Does freddie have some Award? YES (has Grammy)
   - Therefore: freddie is AUTOMATICALLY classified as AwardWinningArtist
   
   The reasoner adds this new fact without human intervention:
   ```python
   freddie.is_a = [Artist, AwardWinningArtist]
   ```

### Why this "Intelligent" (_I prefer "Automated"_) approach to data organization is cool 

1. AUTOMATIC DISCOVERY:
   - System discovers new facts from existing data + rules
   - No need to manually maintain derived classifications
   - As data changes, classifications update automatically

2. CONSISTENCY MAINTENANCE:
   - If artist loses awards, they're automatically reclassified
   - If new restriction rules are added, they apply to all existing data
   - Impossible to have inconsistent derived data

3. LOGICAL INFERENCE:
   - Complex multi-step reasoning chains
   - If A implies B and B implies C, then A implies C (transitivity)
   - Much more powerful than simple database queries

4. KNOWLEDGE REUSE:
   - Same restriction rules work across different datasets
   - Rules encode domain expertise that can be shared
   - New data automatically inherits accumulated knowledge

#### Example Workflow 

STEP 1 - Define Rules:
```python
class CollaborativeArtist(Artist):
    equivalent_to = [Artist & collaboratesWith.some(Artist)]
```

STEP 2 - Add Data:
```python
john = Artist("John")
paul = Artist("Paul") 
john.collaboratesWith = [paul]
```

STEP 3 - Run Reasoner:
```python
sync_reasoner_hermit()
# or our homemade, handcrafted DIY reasoner...
```

STEP 4 - Automatic Results:
```python
# System automatically infers:
john.is_a = [Artist, CollaborativeArtist]
# Without any manual classification!
```

### Contrast with Traditional/Manual Approach 

MANUAL (Traditional):
- Add john and paul to artists table
- Add collaboration to collaborations table  
- Remember to add john to collaborative_artists table
- Remember to update if collaborations change
- Write application code to maintain consistency

AUTOMATIC (OWL):
- Define restriction once: CollaborativeArtist = Artist & collaboratesWith.some(Artist)
- Add basic facts: john collaboratesWith paul
- Reasoner automatically handles all derived classifications
- Changes propagate automatically
- No custom application logic needed

So, Ontologies (OWL in our case) enable "intelligent" (_"automated"_?) data organization - the system can reason
about your data using domain knowledge encoded as logical rules, discovering new facts and maintaining consistency without constant human intervention.

# Serializing Music Ontology Data

In [79]:
# Save the extended ontology
extended_onto.save(file="toy_music_extended_owlready.owl", format="rdfxml")
extended_onto.save(file="toy_music_extended_owlready.ttl", format="turtle")

In [80]:
# Save only new classes (create a focused ontology)
focused_onto = get_ontology("http://example.org/music_focused.owl")
with focused_onto:
    # Import specific classes from the extended ontology
    class Playlist(extended_onto.Playlist):
        pass

    class Concert(extended_onto.Concert):
        pass

    class MusicVideo(extended_onto.MusicVideo):
        pass


focused_onto.save(file="toy_music_focused_owlready.owl")

In [81]:
# Export to different formats
print("Ontology saved in multiple formats:")
print(f"RDF/XML: toy_music_extended_owlready.owl")
print(f"Turtle: toy_music_extended_owlready.ttl")
print(f"Focused: toy_music_focused_owlready.owl")

Ontology saved in multiple formats:
RDF/XML: toy_music_extended_owlready.owl
Turtle: toy_music_extended_owlready.ttl
Focused: toy_music_focused_owlready.owl


In [82]:
# Print ontology statistics
print(f"\nOntology statistics:")
print(f"Classes: {len(list(extended_onto.classes()))}")
print(f"Properties: {len(list(extended_onto.properties()))}")
print(f"Individuals: {len(list(extended_onto.individuals()))}")


Ontology statistics:
Classes: 14
Properties: 9
Individuals: 9


In [83]:
# Destroy ontology (clean up remember?, also helpful if the ontology is too big...)
# extended_onto.destroy()

Different serialization formats serve different purposes. 
* Turtle is human-readable for debugging
* XML integrates with OWL tools
* JSON-LD works with web applications.
* Separating new classes helps with modular ontology development.

.